# Linear programming with PuLP - A Quick Start Guide

From tutorial: https://www.youtube.com/watch?v=mGPMrYnxIcQ

**Maximize** (e.g. profit)
- $z = 10*x_1 + 20*x_2$

**Subject to the constrains**
- $-x_1 + 2*x_2 \leq 15$
- $x_1 + x_2 \leq 12$
- $5*x_1 + 3*x_2 \leq 45$
- $x_1, x_2 \geq 0$

In [29]:
# imports
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [30]:
# Initialize the LP problem
model = LpProblem("My_Maximization_Problem", LpMaximize)
model

My_Maximization_Problem:
MAXIMIZE
None
VARIABLES

In [31]:
# Decision variables
x = LpVariable.dicts("x", [1,2], lowBound=0, cat='Continuous') # 2 variables, both greater than 0
x

{1: x_1, 2: x_2}

In [32]:
# Objective function
obj_func = 10*x[1] + 20*x[2] # 10x1 + 20x2
print(obj_func, "\n")

model += obj_func # add objective function to the model
print(model)

10*x_1 + 20*x_2 

My_Maximization_Problem:
MAXIMIZE
10*x_1 + 20*x_2 + 0
VARIABLES
x_1 Continuous
x_2 Continuous



In [33]:
# Constraints
model += (-x[1] + 2*x[2] <= 15, "price_constraint") # -x1 + 2*x2 <= 15
model += (x[1] + x[2] <= 12, "manpower_constraint") # x1 + x2 <= 12
model += (5*x[1] + 3*x[2] <= 45, "distance_constraint") # 5*x1 + 3*x2 <= 45
# x1, x2 >= 0 (already defined in the decision variables)
model

My_Maximization_Problem:
MAXIMIZE
10*x_1 + 20*x_2 + 0
SUBJECT TO
price_constraint: - x_1 + 2 x_2 <= 15

manpower_constraint: x_1 + x_2 <= 12

distance_constraint: 5 x_1 + 3 x_2 <= 45

VARIABLES
x_1 Continuous
x_2 Continuous

.solve() calls the underlying solver, modifies the model object, and returns the integer status of the solution, which will be 1 if the optimum is found.   
 For the rest of the status codes, see [LpStatus](https://www.coin-or.org/PuLP/constants.html#pulp.constants.LpStatus).

 
| LpStatus key         | string value  | numerical value |
|----------------------|---------------|-----------------|
| LpStatusOptimal      | "Optimal"     | 1               |
| LpStatusNotSolved    | "Not Solved"  | 0               |
| LpStatusInfeasible   | "Infeasible"  | -1              |
| LpStatusUnbounded    | "Unbounded"   | -2              |
| LpStatusUndefined    | "Undefined"   | -3              |

You can get the optimization results as the attributes of model. The function value() and the corresponding method .value() return the actual values of the attributes:

model.objective holds the value of the objective function, model.constraints contains the values of the slack variables, and the objects x and y have the optimal values of the decision variables. model.variables() returns a list with the decision variables:

In [36]:
# Solve the optimization problem
status = model.solve()
status

print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in model.variables():
    print(f"{var.name}: {var.value()}")
for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 210.0
x_1: 3.0
x_2: 9.0
price_constraint: 0.0
manpower_constraint: 0.0
distance_constraint: -3.0


In [38]:
print(model.variables())
print(model.variables()[0] is x[1])
print(model.variables()[1] is x[2])

[x_1, x_2]
True
True


In [43]:
# Check which solver is being used/called by PuLP
model.solver

The output informs you that the solver is CBC. You didn’t specify a solver, so PuLP called the default one.

If you want to run a different solver, then you can specify it as an argument of .solve(). For example, if you want to use GLPK and already have it installed, then you can use ```solver=GLPK(msg=False)``` in the last line. 
Keep in mind that you’ll also need to import it.  When you have GLPK imported, you can use it inside .solve():